In [2]:
import pandas as pd

In [25]:
# Packages
from bs4 import BeautifulSoup as bs
import requests

def fundamental_metric(soup, metric):
    return soup.find(text = metric).find_next(class_='snapshot-td2').text

def get_fundamental_data(df):
    for symbol in df.index:
        try:
            url = ("http://finviz.com/quote.ashx?t=" + symbol.lower())
            headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:20.0) Gecko/20100101 Firefox/20.0'}
            soup = bs(requests.get(url, headers=headers).content)
            for m in df.columns:
                df.loc[symbol, m ] = fundamental_metric(soup, m)
        except Exception as e:
            print(symbol, 'not found')
    return df

#############################
# Select Stocks and Metrics #
#############################
stock_list = [
    'FB', 'AMZN', 'AAPL', 'MSFT', 'GOOG','PG','KO','IBM','DG','XOM','PEP', 'DKS', 'RTX','BRK-A']

metric = ['Price',
'P/B',
'P/E',
'Forward P/E',
'PEG',
'Debt/Eq',
'EPS (ttm)',
'Dividend %',
'ROE',
'ROI',
'EPS Q/Q',
'Insider Own'
]

df = pd.DataFrame(index = stock_list, columns = metric)
df = get_fundamental_data(df)

for char in ["%", " ", "/"]:
    df.columns = df.columns.str.replace(char, "")

for col in ['Dividend', 'ROE', 'ROI', 'EPSQQ', 'InsiderOwn']:
    df[col] = df[col].str.replace("%", '')
df = df.apply(pd.to_numeric, errors='coerce')

In [26]:
from sklearn.preprocessing import StandardScaler

features = ['PE', 'PB', 'InsiderOwn', 'EPS(ttm)']

x = df.loc[:, features].values

y = df.loc[:, 'Price'].values

x = StandardScaler().fit_transform(x)

In [27]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
principal_components = pca.fit_transform(x)

principal_df = pd.DataFrame(data = principal_components, columns = ['principal component 1', 'principal component 2'])

In [28]:
final_df = pd.concat([principal_df, df[['Price']]], axis = 1)

In [31]:


fig = plt.figure(figsize = (8,8))
ax = fig.addsubplot(1,1,1)
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 Component PCA', fontsize=20)


AttributeError: module 'matplotlib' has no attribute 'figure'